# 🛒 E-Commerce Analytics Project (Pandas + PySpark)
---
This notebook performs data analysis on `ecommerce_sample500.csv`, replicating the structure of the *Employees Office Data Analytics* PDF.

**Steps Covered:**
1. Import libraries
2. Load and explore dataset
3. Perform Pandas-based EDA
4. Visualization with Matplotlib
5. PySpark-based distributed analytics
6. Export results


In [ ]:

import os
import pandas as pd
import matplotlib.pyplot as plt
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum as _sum, avg as _avg, countDistinct, to_date


In [ ]:

# Load dataset
csv_path = "ecommerce_sample500.csv"  # Update path if needed
df = pd.read_csv(csv_path)

# Convert InvoiceDate to datetime
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'], errors='coerce')

# Add computed column
df['Revenue'] = df['Quantity'] * df['UnitPrice']

df.head()


In [ ]:

print("Dataset shape:", df.shape)
print("\nMissing values per column:\n", df.isnull().sum())
print("\nBasic statistics:\n", df.describe(include='number').T)


In [ ]:

# Basic KPIs
total_orders = df['InvoiceNo'].nunique()
unique_customers = df['CustomerID'].nunique()
total_revenue = df['Revenue'].sum()
avg_order_value = df.groupby('InvoiceNo')['Revenue'].sum().mean()

print(f"Total Orders: {total_orders}\nUnique Customers: {unique_customers}\nTotal Revenue: {total_revenue:.2f}\nAOV: {avg_order_value:.2f}")

# Top products by revenue
top_products = df.groupby(['StockCode','Description'])['Revenue'].sum().reset_index().sort_values('Revenue', ascending=False)
top_products.head(10)


In [ ]:

out_dir = "outputs"
os.makedirs(out_dir, exist_ok=True)

# Quantity distribution
plt.figure(figsize=(8,5))
plt.hist(df['Quantity'], bins=20, color='skyblue', edgecolor='black')
plt.title('Quantity Distribution')
plt.xlabel('Quantity')
plt.ylabel('Count')
plt.savefig(f"{out_dir}/quantity_distribution.png")
plt.show()

# Revenue by top 10 countries
country_sales = df.groupby('Country')['Revenue'].sum().sort_values(ascending=False).head(10)
plt.figure(figsize=(8,5))
country_sales.plot(kind='bar', color='coral', edgecolor='black')
plt.title('Top 10 Countries by Revenue')
plt.xlabel('Country')
plt.ylabel('Total Revenue')
plt.savefig(f"{out_dir}/top_countries_revenue.png")
plt.show()


In [ ]:

spark = SparkSession.builder.appName("EcommerceAnalysis").master("local[*]").getOrCreate()

spark_df = spark.read.option("header", True).option("inferSchema", True).csv(csv_path)
spark_df = spark_df.withColumn("Revenue", col("Quantity") * col("UnitPrice"))

spark_df.printSchema()


In [ ]:

# Spark Aggregations
agg_summary = spark_df.agg(
    _sum("Revenue").alias("total_revenue"),
    countDistinct("InvoiceNo").alias("total_orders"),
    countDistinct("CustomerID").alias("unique_customers")
).collect()[0]

print(f"Total Revenue: {agg_summary['total_revenue']:.2f}\nTotal Orders: {agg_summary['total_orders']}\nUnique Customers: {agg_summary['unique_customers']}")


In [ ]:

# Top products by revenue (Spark)
top_products_spark = spark_df.groupBy("StockCode", "Description").agg(_sum("Revenue").alias("Revenue")).orderBy(col("Revenue").desc())
top_products_spark.show(10)

# Country-level revenue
country_sales_spark = spark_df.groupBy("Country").agg(_sum("Revenue").alias("TotalRevenue"), _avg("Revenue").alias("AvgRevenue")).orderBy(col("TotalRevenue").desc())
country_sales_spark.show(10)


In [ ]:

# Save outputs
output_dir = "outputs_spark"
os.makedirs(output_dir, exist_ok=True)

# Fix for Windows Hadoop native I/O issue
spark.conf.set("spark.hadoop.io.nativeio.enabled", "false")

# Write outputs safely
top_products_spark.coalesce(1).write.option("header", True).mode("overwrite").csv(f"{output_dir}/top_products")
country_sales_spark.coalesce(1).write.option("header", True).mode("overwrite").csv(f"{output_dir}/country_sales")

spark.stop()
print(f"✅ Results saved in '{output_dir}' folder.")
